# Transformers

Blog to read: https://jalammar.github.io/illustrated-transformer/

## Introduction

If we remember, in the previous notebook we discussed about the Attention model using Bi-directional LSTM, but we identified some drawbacks where in, it is not capable of handling large length of sentences. Now, we are going to discuss about the extension of Attention models using Transformers

The Encoder-Decoders implemented with Bi-directional LSTM has the main drawback where it failed to capture the context in the long sentences. Now, let us explore about the Transformers using the use case of "Langugage Translation".

## High-level Architecture of Transformers

Consider the input sentence is of French language and the output language is English. French language is passed into the TRANSFORMER and its giving the output as English. 

French -> TRANSFORMER -> English

But curious to know what is inside Transformer, so let us break down into the next level. Inside the transformer, we have a set of Encoders and Decoders clubbed together. The output of the encoders is passed to the decoders and the final output of the English language is provided by Decoders. 

Again digging deeper, there is a set of 6 Encoders stacked together and the final output of the last encoder is passed to the each and every decoder present.  

Now, that we discussed about the representation let us focus on especially what is inside the Encoder and Decoder. 

### Encoder components

Inside the Encoder, it is divided into two layers:

1. Self Attention layer 

2. Feed forward Neural Network

### Decoder components

Inside the Decoder, it is divided into three layers:

1. Self Attention layer 

2. Encoder-decoder Attention layer 

2. Feed forward Neural Network

## Understanding operations in Encoder

1. Input sentence gets broken into words and each and every word gets converted into embeddings using any of the word embedding techniques. (Remember the encoding/embedding happens only at the bottom-most encoder and it will be of size 512)

2. All the embeddings gets passed parallely to the Encoder in form of inputs (x1, x2, x3, ...) and the output of the first layer (Self attention layer) will be in form of (z1, z2, z3, ...).

3. The same is passed into the Feed forward Neural Network and the output will be in form of (r1, r2, r3, ...).

**Note:** Each input word passes in its own path inside the encoder, Self-attention layer has some dependencies inside but there are no dependencies inside the feed forward layer.

Now, its time to explore how the input words (x1, x2, ...) are getting converted to z1, z2, z3, ... inside the Self-Attention layer.

To understand the same lets consider an example of two words - "Thinking" and "Machines".

### Self-Attention layer operations

Before moving further, let us look at an example sentence: 

*"The animal didn't cross the street because **it** was too tired"*

Here if we ask a question like what does the word "it" refer to? The answer is very simple its referring to animal. But will our model can predict the same? 

Initially when the model is processing "it", then it already has the context of the previous processed words, so it will focus more on "animal" and "street". But since it relates more to the word "animal" it goes and associates with the same.

Here, Self-attention plays a major role and it helps the model focus on the right words leading to a better understanding leading to right translations. 

Let us understand step-by-step what is exactly happening inside the Self attention layer. For ease, lets consider there are only two words. "Thinking" and "Machines"

#### Step 1 - Creation of word embeddings and corresponding Query, Key and Value vectors

We need to first convert the above both words into corresponding vector embeddings with a size of 512. We will also create three vectors - Query, Key and Value vectors corresponding to the input words. 

Thinking - Q1, K1 and V1 

Machines - Q2, K2 and V2 

These three vectors are computed by multiplying the embeddings by three matrices that we trained during the training process. These three vectors will have a dimensionality of 64 whereas the input sentences are of 512. 



#### Step 2 - Computation of Query * Key score

Consider we are calculating the score of the first word "Thinking" then we need to compute the score for each word in the given sentence aganist this word "Thinking". So that we will be able to know how much this word relates to the other words in the sentence. 

In short, we need to score this word with each other word in the given sentence. It determines how much focus we need to place on other words in the given sentence. 

Here, let us calculate the score for the word "Thinking" aganist all the other words in the input sentence. 

Q1 * K1 = 112

Q1 * K2 = 96 ... so on and so forth.

#### Step 3 - Divide score by square root of the dimension of the key vectors $\sqrt(d_k)$ – 64

Once the score is computed aganist each word in the input sentence, we need to divide the same with the square root of the key vector dimension. This applies for all the scores calculated in the previous step. 

Q1 * K1 = 112/8 = 14 

Q1 * K2 = 96/8 = 12 

This division with square root helps us to get stable gradients. 

#### Step 4 - Softmax operation

Post division, once we pass to softmax operation. It applies the normalization and computes the scores in such a way like it adds upto 1 and converts all the values into positive. 

This Softmax score determines how much each word will be expressed at this position. Its very clear the position where the word is present will have the highest softmax score but it sometimes it helps us to determine how much it relates to its relevant word. 

#### Step 5 - Multiply Softmax score with corresponding Value Vector

We will now be multiplying the computed softmax score with the value vector. We are doing this to remove the irrelevant words automatically. Hence, for example for one of the word the softmax score is 0.001 then if we multiply with its value vector then the words gets eliminated. 

At the final we will only have the relevant words and the remaining words which are irrelevant will be worned out. 

#### Step 6 - Summing up of weighted value vectors

After each value vector has been scaled by its attention weight, the final step is simply summing up all the weighted vectors. This sum represents the new representation of the word at this position, capturing context from the entire sequence.

The final output is represented in the symbol of 'z'.

All the above steps are performed parallely for all the input words in the sequence, allowing the model to generate contextualized embeddings for each token efficiently. The final vector (z) will be sent to the Feed forward layer in the Encoder. 

### Matrix Multiplication

In the above operations we discussed in simple mathematics, but in real-time the same operatios are performed in matrix to improve the efficiency and for faster computation. 

Lets observe the high-level steps again in for of matrices: 
1. All the input words after converting into embeddings will be stacked together in a single array named 'X'. 

2. Then the X matrix will be multipled with the weight matrices which are calculated during the training (WQ, WK, WV) then we will get the Q, K and V matrices respectively. 

3. Now all the steps till six can be combined into one single formula. The final formula will look like something as below: 

Z vector = Softmax((Query vector * Key vector) / ($\sqrt(d_k)$)) * Value vector 